In [94]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.linear_model import BayesianRidge
import re
import os
import warnings
warnings.filterwarnings("ignore")
s_dir = "./submission_0/"
pd.set_option('display.max_columns', None)
test = pd.read_csv("./datadown/test.csv")
y = pd.read_csv("./dataup/y.csv", )
y_origin = pd.read_csv("./datadown/train.csv", )
nomodel_sub = pd.read_csv("./submission_0/sub_nomodel.csv")

In [95]:
from tensorflow.keras.callbacks import TensorBoard
import datetime

# Define log directory with timestamp
log_dir = "./logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

def scan_and_aggregate_submissions(base_dir):
    # Initialize empty DataFrames for each type
    oof_dfs = {}
    test_dfs = {}
    # Iterate over all directories in the base directory
    for dir_name in os.listdir(base_dir):
        dir_path = os.path.join(base_dir, dir_name)
        # Check if it's a directory
        if (os.path.isdir(dir_path)) & (dir_name.startswith("submission")):
            # Initialize DataFrames for the current directory
            oof_df = pd.DataFrame()
            test_df = pd.DataFrame()
            # Iterate over all files in the directory
            for file_name in os.listdir(dir_path):
                file_path = os.path.join(dir_path, file_name)

                # Check if the file is a CSV and matches the pattern
                if file_name.endswith('.csv'):
                    if re.findall(r"_oof", file_name):
                        oof_df = pd.concat([oof_df, pd.read_csv(file_path)], axis=1)
                    elif re.findall(r"_test", file_name):
                        test_df = pd.concat([test_df, pd.read_csv(file_path)], axis=1)
            # Add a suffix to the columns to indicate the batch
            oof_df.columns = [f"{col}_{dir_name}" for col in oof_df.columns]
            test_df.columns = [f"{col}_{dir_name}" for col in test_df.columns]
            # Store the DataFrames in the dictionaries
            oof_dfs[dir_name] = oof_df
            test_dfs[dir_name] = test_df

    # Aggregate all DataFrames
    sm_oof_agg = pd.concat(oof_dfs.values(), axis=1)
    sm_test_agg = pd.concat(test_dfs.values(), axis=1)

    return sm_oof_agg, sm_test_agg

sm_oof_agg, sm_test_agg = scan_and_aggregate_submissions(s_dir)


In [96]:
oof_all = pd.concat(
    [
        y_origin[["num_sold"]].rename(columns={"num_sold": "num_sold_origin"}),
        y,
        sm_oof_agg,
    ],
    axis=1,
)
oof_all

,num_sold_origin,num_sold,BRidge_submissions_5,CAT3_submissions_5,CAT5_submissions_5,CAT_submissions_5,HGB2_submissions_5,HGB3_submissions_5,HGB4_submissions_5,HGB_submissions_5,LGBM2_submissions_5,LGBM3_submissions_5,LGBM4_submissions_5,LGBM5_submissions_5,LGBM6_submissions_5,LGBM7_submissions_5,LR_submissions_5,NN1_submissions_5,NN2_submissions_5,NN3_submissions_5,Ridge_submissions_5,XGB2_submissions_5,XGB3_submissions_5,XGB4_submissions_5,XGB5_submissions_5,YDF_submissions_5
0,NaN,4.953125,5.055196,4.976988,4.990742,5.000138,4.970669,4.959844,4.970045,4.973628,4.957910,4.929076,4.958554,4.964052,4.951809,4.948767,5.052416,4.881492,4.760598,5.032609,5.092650,5.009651,4.999286,5.012230,5.012504,4.970455
1,973.0,6.882812,6.869520,6.803995,6.814136,6.796185,6.818586,6.803796,6.810877,6.819162,6.784108,6.752838,6.793062,6.802682,6.801034,6.787491,6.866742,6.758991,6.688148,6.821351,6.906710,6.798010,6.814610,6.822582,6.827233,6.803455
2,906.0,6.808594,6.768643,6.684072,6.700735,6.675063,6.703292,6.694917,6.695851,6.703683,6.653682,6.637265,6.667082,6.689096,6.681265,6.661387,6.765865,6.650045,6.597801,6.702089,6.805870,6.676704,6.692004,6.701296,6.703520,6.678157
3,423.0,6.050781,6.054719,6.016685,6.028953,6.060366,6.046490,6.033805,6.039830,6.045817,5.996015,5.966634,5.997504,6.006598,6.001388,5.992826,6.051940,5.979044,5.842335,6.012423,6.092078,6.070922,6.072653,6.077210,6.077696,6.027930
4,491.0,6.199219,6.321222,6.170931,6.170106,6.193411,6.179232,6.162637,6.172481,6.178086,6.147815,6.129021,6.152480,6.162489,6.165386,6.135647,6.318443,6.124265,5.981115,6.146773,6.358559,6.202479,6.200631,6.209573,6.209544,6.177560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230125,466.0,6.144531,5.868612,6.084982,6.091216,6.096135,6.106352,6.101238,6.092536,6.103722,6.017305,6.021604,6.013556,5.974486,5.997832,6.001863,5.868770,5.855519,5.861982,5.920529,5.809962,6.078082,6.077152,6.076802,6.075336,6.080376
230126,2907.0,7.976562,7.861974,7.892147,7.897832,7.939498,7.945387,7.952396,7.929972,7.941422,7.862293,7.876986,7.874396,7.825332,7.862341,7.854038,7.862128,7.641141,7.817236,7.822078,7.804809,7.965805,7.960750,7.963765,7.959655,7.922291
230127,2299.0,7.742188,7.540897,7.658772,7.659366,7.691973,7.702058,7.698463,7.691830,7.704554,7.583570,7.621325,7.597151,7.587528,7.616158,7.589478,7.541052,7.477164,7.555460,7.615555,7.483374,7.705388,7.701572,7.699794,7.696357,7.738010
230128,1242.0,7.125000,6.898285,7.112733,7.124053,7.154492,7.142194,7.130150,7.129292,7.140648,7.036203,7.051366,7.039867,7.023267,7.040671,7.038988,6.898441,7.041482,7.016694,6.990996,6.840146,7.144080,7.147243,7.138192,7.133782,7.128285


In [97]:
oof_all['num_sold_origin'].fillna(oof_all['num_sold'], inplace=True)

In [98]:
test_all = sm_test_agg.copy(True)

In [99]:
test_all

,BRidge_submissions_5,CAT3_submissions_5,CAT5_submissions_5,CAT_submissions_5,HGB2_submissions_5,HGB3_submissions_5,HGB4_submissions_5,HGB_submissions_5,LGBM2_submissions_5,LGBM3_submissions_5,LGBM4_submissions_5,LGBM5_submissions_5,LGBM6_submissions_5,LGBM7_submissions_5,LR_submissions_5,NN1_submissions_5,NN2_submissions_5,NN3_submissions_5,Ridge_submissions_5,XGB2_submissions_5,XGB3_submissions_5,XGB4_submissions_5,XGB5_submissions_5,YDF_submissions_5
0,4.998052,4.950604,4.963764,4.988036,4.963485,4.952037,4.966473,4.966921,4.927054,4.932944,4.928423,4.934938,4.945224,4.931764,4.999428,4.745428,4.648595,4.395978,4.946635,4.987809,4.979076,4.982833,4.983639,4.960856
1,6.935839,6.820765,6.836245,6.837700,6.813676,6.823747,6.814553,6.816463,6.793692,6.803356,6.806989,6.803409,6.818374,6.800581,6.937215,6.660741,6.709018,6.836604,6.884230,6.838588,6.831857,6.837193,6.837594,6.826388
2,6.533808,6.546316,6.552676,6.562584,6.535932,6.538859,6.538990,6.539057,6.523429,6.532944,6.532189,6.531357,6.539062,6.532437,6.535185,6.341456,6.459890,6.493037,6.482256,6.552596,6.545718,6.550402,6.551767,6.544566
3,6.006999,6.017890,6.018560,6.047549,6.004227,6.015252,6.006468,6.006857,5.990121,6.005228,5.990044,5.997787,6.013825,5.998052,6.008376,5.767015,5.890791,5.904617,5.955516,6.039769,6.038361,6.035953,6.036731,6.003813
4,6.269510,6.146058,6.148205,6.176049,6.140720,6.145890,6.144433,6.143315,6.147680,6.139400,6.149976,6.151155,6.154863,6.150985,6.270886,5.892844,6.042407,6.065396,6.218013,6.168029,6.163730,6.166368,6.166305,6.172931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98545,5.503418,5.970506,5.973355,5.956447,5.962600,5.954905,5.952890,5.963178,5.959079,5.938867,5.954449,5.952507,5.943346,5.951212,5.510264,5.979268,5.734654,5.850679,5.260140,5.960455,5.949422,5.954118,5.954607,5.966519
98546,7.453045,7.823137,7.825996,7.817033,7.847985,7.835806,7.837523,7.852043,7.796944,7.817928,7.807743,7.809929,7.826960,7.807891,7.459886,7.927389,7.737283,7.613555,7.210986,7.832370,7.837391,7.832678,7.833242,7.792972
98547,7.314123,7.685364,7.684637,7.681921,7.701651,7.688603,7.692930,7.704247,7.653188,7.666553,7.667678,7.684306,7.674638,7.659074,7.320965,7.744124,7.597712,7.526944,7.071884,7.686186,7.690324,7.684986,7.683971,7.734067
98548,6.552756,7.061832,7.074555,7.078335,7.059225,7.041407,7.048791,7.059933,7.016774,7.028722,7.024441,7.031723,7.031981,7.021231,6.559600,7.051770,6.912066,6.957425,6.309920,7.063213,7.064009,7.057796,7.058531,7.065208


In [107]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_absolute_percentage_error

class Ensemble:
    def __init__(self, n_splits=5, random_state=42):
        self.n_splits = n_splits
        self.state = random_state
        self.scores = [] 
        
    def ensemble(self, X, y, test):
        oof_pred = np.zeros(X.shape[0]) 
        test_pred = np.zeros(test.shape[0]) 
        feature_importances = np.zeros(X.shape[1])
        model = BayesianRidge() 
        kf = KFold(n_splits=self.n_splits, random_state=self.state, shuffle=True) 
 
        for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X, y)): 
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx] 
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx] 
            model.fit(X_train, y_train) 
            # Calculate feature importances using coefficients
            feature_importances += np.abs(model.coef_) / self.n_splits
            y_pred_probs = model.predict(X_val) 
            oof_pred[val_idx] = y_pred_probs 
            test_pred += model.predict(test) / self.n_splits 
 
            score = mean_absolute_percentage_error(y_val, y_pred_probs) 
            self.scores.append(score) 
             
        # Normalize feature importances
        feature_importances = feature_importances / np.sum(feature_importances)
        
        return {
            'mean_score': np.mean(self.scores), 
            'oof_pred': oof_pred, 
            'test_pred': test_pred,
            'feature_importances': feature_importances
        }

# Usage example
e = Ensemble()

submission_ = e.ensemble(
    oof_all.drop(columns=["num_sold_origin", "num_sold"]),
    oof_all[['num_sold']],
    test_all.iloc[: test.shape[0], :],
)
feature_importances = submission_['feature_importances']

# To display feature importances
for name, importance in zip(
    oof_all.drop(columns=["num_sold_origin", "num_sold"]).columns, feature_importances
):
    print(f"{name}: {importance:.4f}")

BRidge_submissions_5: 0.4365
CAT3_submissions_5: 0.0150
CAT5_submissions_5: 0.0130
CAT_submissions_5: 0.0019
HGB2_submissions_5: 0.0012
HGB3_submissions_5: 0.0011
HGB4_submissions_5: 0.0022
HGB_submissions_5: 0.0030
LGBM2_submissions_5: 0.0012
LGBM3_submissions_5: 0.0029
LGBM4_submissions_5: 0.0032
LGBM5_submissions_5: 0.0041
LGBM6_submissions_5: 0.0068
LGBM7_submissions_5: 0.0051
LR_submissions_5: 0.4343
NN1_submissions_5: 0.0003
NN2_submissions_5: 0.0006
NN3_submissions_5: 0.0000
Ridge_submissions_5: 0.0023
XGB2_submissions_5: 0.0049
XGB3_submissions_5: 0.0134
XGB4_submissions_5: 0.0299
XGB5_submissions_5: 0.0128
YDF_submissions_5: 0.0045


In [101]:
sb_allmodel = pd.read_csv("./submission_0/submissions_5/submission_allmodel.csv")


array([5.00426935, 6.87515474, 6.58465344, ..., 7.91353444, 7.2817989 ,
       7.39822843])

In [102]:
sub = pd.concat([nomodel_sub, 1.01* np.expm1(sb_allmodel)], axis = 1)
sub

,id,num_sold,XGB2,XGB3,XGB4,XGB5,LGBM2,LGBM3,LGBM4,LGBM5,LGBM6,LGBM7,Ensemble
0,230130,158.0,147.070942,145.783426,146.335966,146.454775,138.342222,139.165348,138.533116,139.445220,140.897316,139.000099,135.765122
1,230131,1031.0,941.492084,935.169312,940.178869,940.556362,900.113258,908.864130,912.175997,908.912732,922.631458,906.342601,882.883395
2,230132,775.0,707.062104,702.208765,705.510580,706.475639,686.708484,693.282863,692.759077,692.182077,697.543666,692.931094,672.341416
3,230133,458.0,422.983884,422.387660,421.369344,421.697872,402.447741,408.588926,402.416305,405.552456,412.125277,405.659969,402.053257
4,230134,523.0,481.007119,478.939088,480.207250,480.176501,471.297381,467.402764,472.383185,472.941836,474.702480,472.860968,464.845627
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98545,328675,475.0,390.654392,386.356910,388.180263,388.370818,390.115973,382.289613,388.308933,387.553847,384.010391,387.050830,379.247885
98546,328676,2924.0,2545.095793,2557.911938,2545.880878,2547.316965,2456.477388,2508.588942,2483.157550,2488.595576,2531.359166,2483.525761,2473.928270
98547,328677,2598.0,2198.822788,2207.944556,2196.183912,2193.955770,2127.416719,2156.053841,2158.483097,2194.689505,2173.564744,2139.981283,2168.685802
98548,328678,1331.0,1178.864824,1179.804024,1172.489784,1173.353562,1125.324592,1138.863362,1133.993496,1142.288828,1142.583860,1130.356056,1136.330793


array([ 148.04814091,  966.92513188,  722.90013164, ..., 2733.03670076,
       1452.60056152, 1632.08873456])

In [110]:
sb

,id,num_sold
0,230130,144.0
1,230131,975.0
2,230132,732.0
3,230133,424.0
4,230134,500.0
...,...,...
98545,328675,504.0
98546,328676,3205.0
98547,328677,2984.0
98548,328678,1520.0


In [111]:
import matplotlib.pyplot as plt
plt.scatter(sb['NN3'], sb['num_sold'], label='num_sold', alpha=0.5)
# plt.scatter(nomodel_sub['id'], nomodel_sub['pred'], label='pred', alpha=0.5,s = 1)
plt.legend()
plt.show()


KeyError: 'NN3'

In [115]:
sb = pd.read_csv("./datadown/sample_submission.csv")
sb['num_sold'] = np.round(np.clip(1.01*np.expm1(submission_['test_pred']), 0, 6000))
sb

,id,num_sold
0,230130,150.0
1,230131,977.0
2,230132,730.0
3,230133,431.0
4,230134,503.0
...,...,...
98545,328675,484.0
98546,328676,3096.0
98547,328677,2760.0
98548,328678,1467.0


In [117]:
sb.to_csv("./submission_0/submission.csv", index=None)
